# Policy refinement with human feedback (Policy v2 → v3)

이 노트북은 두 가지를 해준다:

1. **human_preferred_decision 컬럼이 포함된 피드백 테이블 템플릿 만들기**
   - 입력: `../results/disagreement_cases_with_text.csv`
   - 출력: `../results/disagreement_with_human_template.csv`
   - 여기에 사람이 `human_preferred_decision`을 채워 넣는다.

2. **사람 피드백 + 기존 정책 설명을 바탕으로, LLM에게 Policy A/B/C v3 초안을 요청**
   - 입력:
     - 사람이 라벨링한 `disagreement_with_human_filled.csv` (파일명은 아래에서 지정)
     - 기존 Policy A/B/C 설명 텍스트
   - 출력:
     - 터미널 출력 + `../results/policy_v3_suggestions.md` 파일


In [1]:
import os
from pathlib import Path
import pandas as pd
import textwrap

from dotenv import load_dotenv
from openai import OpenAI

# 프로젝트 디렉토리 구조 가정
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"

DISAGREE_CSV = RESULTS_DIR / "disagreement_cases_with_text.csv"

print("PROJECT_ROOT :", PROJECT_ROOT)
print("RESULTS_DIR  :", RESULTS_DIR)
print("DISAGREE_CSV :", DISAGREE_CSV)

# Load environment variables (expects .env with OPENAI_API_KEY)
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", ""))


PROJECT_ROOT : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2
RESULTS_DIR  : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2\results
DISAGREE_CSV : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2\results\disagreement_cases_with_text.csv


In [2]:
# 1) 사람 피드백용 템플릿 만들기

if not DISAGREE_CSV.exists():
    raise FileNotFoundError(f"{DISAGREE_CSV} 가 없습니다. 먼저 분석 노트북을 돌려 disagreement_cases_with_text.csv를 생성하세요.")

df_dis = pd.read_csv(DISAGREE_CSV)

print("원본 disagreement_cases_with_text.csv 예시:")
display(df_dis.head())

# human_preferred_decision 컬럼 추가 (빈 값)
if "human_preferred_decision" not in df_dis.columns:
    df_dis["human_preferred_decision"] = ""

# 설명용 컬럼도 하나 추가할 수 있음 (선택)
if "human_comment" not in df_dis.columns:
    df_dis["human_comment"] = ""

TEMPLATE_PATH = RESULTS_DIR / "disagreement_with_human_template.csv"
df_dis.to_csv(TEMPLATE_PATH, index=False, encoding="utf-8-sig")

print("\n사람 라벨링 템플릿을 저장했습니다:")
print(" ->", TEMPLATE_PATH)
print("\n이 파일을 엑셀/VSCode 등으로 열어서 다음 컬럼을 채우세요:")
print(" - human_preferred_decision: BLOCK / WARN_AND_ALLOW / ALLOW 중 하나")
print(" - human_comment (선택): 왜 그렇게 판단했는지 간단 설명")


원본 disagreement_cases_with_text.csv 예시:


,sample_id,A,B,C,thread_id,role,order_in_thread,text
0,26,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,5,post,0,전전 3.6졸이면 ㅁㅌㅊ임? 전공 성적은 3.9 넘고 기초랑 교양에서 많이 까먹긴함
1,30,BLOCK,BLOCK,WARN_AND_ALLOW,5,comment,4,4.1졸이면 ㅁㅌㅊ임?
2,31,BLOCK,BLOCK,WARN_AND_ALLOW,5,comment,5,이건 걍 ㅅㅌㅊ아님?ㄷㄷ
3,53,BLOCK,BLOCK,WARN_AND_ALLOW,7,comment,1,빌넣 받아주셔요
4,80,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,10,comment,8,헌급방은 빡셈



사람 라벨링 템플릿을 저장했습니다:
 -> C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2\results\disagreement_with_human_template.csv

이 파일을 엑셀/VSCode 등으로 열어서 다음 컬럼을 채우세요:
 - human_preferred_decision: BLOCK / WARN_AND_ALLOW / ALLOW 중 하나
 - human_comment (선택): 왜 그렇게 판단했는지 간단 설명


## 2) 사람이 할 일

1. 방금 생성된 `disagreement_with_human_template.csv` 파일을 엽니다.
2. 각 row에 대해:
   - **human_preferred_decision**: `BLOCK`, `WARN_AND_ALLOW`, `ALLOW` 중 하나를 입력
   - **human_comment**: (선택) "자기비하라서 허용", "성기 언급은 최소 경고" 등 이유를 적어도 좋음
3. 수정한 파일을 예를 들어 `disagreement_with_human_filled.csv` 같은 이름으로 저장합니다.
   - 템플릿 파일을 그대로 덮어써도 되지만,
   - 원본을 남기고 싶으면 새 파일 이름을 쓰세요.

아래 셀에서는, 사람이 채운 CSV 파일 경로를 `FILLED_PATH` 변수로 지정해서 사용합니다.


In [3]:
# 3) 사람이 채운 피드백 파일 불러오기

# 사람이 채운 파일 이름을 여기서 지정하세요.
FILLED_PATH = RESULTS_DIR / "disagreement_with_human_filled.csv"  # 예시 이름

if not FILLED_PATH.exists():
    raise FileNotFoundError(
        f"{FILLED_PATH} 가 없습니다. disagreement_with_human_template.csv를 채운 뒤, "
        "이름을 disagreement_with_human_filled.csv 로 저장하거나, 위 경로를 원하는 이름으로 바꾸세요."
    )

df_feedback = pd.read_csv(FILLED_PATH)

print("사람 피드백이 포함된 데이터 예시:")
display(df_feedback.head())

# human_preferred_decision이 비어있는 row는 제외 (필요시)
df_feedback = df_feedback[df_feedback["human_preferred_decision"].notna() & (df_feedback["human_preferred_decision"] != "")]
print(f"유효한 피드백 row 수: {len(df_feedback)}")


사람 피드백이 포함된 데이터 예시:


,sample_id,A,B,C,thread_id,role,order_in_thread,text,human_preferred_decision,human_comment
26,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,5,post,0,전전 3.6졸이면 ㅁㅌㅊ임? 전공 성적은 3.9 넘고 기초랑 교양에서 많이 까먹긴함,ALLOW,ALLOW,NaN
30,BLOCK,BLOCK,WARN_AND_ALLOW,5,comment,4,4.1졸이면 ㅁㅌㅊ임?,ALLOW,ALLOW,NaN
31,BLOCK,BLOCK,WARN_AND_ALLOW,5,comment,5,이건 걍 ㅅㅌㅊ아님?ㄷㄷ,ALLOW,ALLOW,NaN
53,BLOCK,BLOCK,WARN_AND_ALLOW,7,comment,1,빌넣 받아주셔요,ALLOW,ALLOW,NaN
80,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,10,comment,8,헌급방은 빡셈,ALLOW,ALLOW,NaN


유효한 피드백 row 수: 20


In [ ]:
# === NEW: 사람 라벨과 정책 A/B/C 일치율 계산 ============================
policy_cols = ["A", "B", "C"]
stats = []

for p in policy_cols:
    matches = (df_feedback[p] == df_feedback["human_preferred_decision"]).sum()
    total = len(df_feedback)
    acc = matches / total if total > 0 else float("nan")
    stats.append((p, matches, total, acc))

df_stats = pd.DataFrame(stats, columns=["policy", "n_match", "n_total", "accuracy"])

print("\n=== Human–policy agreement summary ===")
print(df_stats)

# 가장 사람과 비슷한 정책 선택
# 전체 데이터 셋이 아닌, 피드백이 있는 샘플에 대해서만 계산된 것임에 유의 (낮은 accuracy가 전체 데이터 셋에 대해서 그렇다는 것이 아님)
best_row = df_stats.sort_values("accuracy", ascending=False).iloc[0]
recommended_policy = best_row["policy"]

print(f"\n현재 사람 라벨과 가장 가까운 정책: Policy {recommended_policy} "
      f"(정확도 {best_row['accuracy']:.3f})")

# LLM 프롬프트에 넣을 수 있도록 텍스트 블록으로 변환
stats_text_lines = []
for _, row in df_stats.iterrows():
    stats_text_lines.append(
        f"- Policy {row['policy']}: "
        f"matches = {row['n_match']} / {int(row['n_total'])}, "
        f"accuracy = {row['accuracy']:.3f}"
    )
stats_block = "\n".join(stats_text_lines)
# ======================================================



=== Human–policy agreement summary ===
  policy  n_match  n_total  accuracy
0      A        4       20      0.20
1      B        9       20      0.45
2      C        0       20      0.00

현재 사람 라벨과 가장 가까운 정책: Policy B (정확도 0.450)


## 4) 기존 Policy A/B/C v2 텍스트 입력

아래 셀의 `POLICY_A_V2`, `POLICY_B_V2`, `POLICY_C_V2` 문자열 안에  
**지금 시뮬레이션에 사용하고 있는 정책 설명(프롬프트용 규칙 텍스트)**를 그대로 붙여 넣으면 됩니다.

- 너무 길어도 괜찮지만, 가능하면 **사람이 읽을 수 있을 정도로 정리된 버전**이 좋습니다.
- 나중에 LLM에게는 이 텍스트와 `df_feedback`를 함께 넘겨서,
  - 정책이 어디서 과도하게 차단/허용하는지
  - 사람 피드백과 더 잘 맞도록 어떻게 수정하면 좋은지
  제안하도록 요청합니다.


In [5]:
# TODO: 여기에 현재 사용중인 A/B/C 정책 설명을 붙여 넣으세요.

POLICY_A_V2 = """Policy A: High Protection, Low Tolerance (매너 최우선)

**Main principles (요약)**  
- 사람·집단을 향한 인신공격·비하·혐오는 최대한 차단한다.  
- 다만, 욕설이 전혀 없고 인신공격도 없는 비판·불만은 그대로 허용한다.  
- 욕설·비속어가 있어도 ‘사람’이 아닌 ‘정책·제도·수업·서비스’만 겨냥하면 원칙적으로 허용하되 경고만 붙인다.  
- 모호한 경우에도, 인신공격·혐오 쪽이면 차단, 단순 감정표현·정책비판이면 허용 쪽으로 본다.  

#### BLOCK 기준

다음에 해당하면 차단한다.

1. **직접적인 인신공격 / 비하**
   - 욕설·비속어·비하 표현이 **개인, 특정 교수, 학생, 댓글 작성자, 소수의 식별 가능한 사람**을 겨냥할 때  
   - 예:  
     - “저 교수 진짜 병x임”, “야 너 같은 멍청이는 학교 오지 마라”  
     - “저년 얼굴 진짜 역겨움”, “틀딱들은 다 꺼져라”  

2. **집단에 대한 혐오·비하**
   - 성별, 지역, 인종, 성적지향, 장애, 종교 등 **사회적 집단 전체**를 모욕·비하하는 표현  
   - 예: “여자는 원래 이래서 안 됨”, “전라도는 다 거짓말쟁이”  

3. **위협·신상털이·괴롭힘 선동**
   - 폭력·자해·살해·성폭력 등을 암시하거나 직접적으로 위협  
   - 신상 공개, 특정인에 대한 집단 괴롭힘 유도  
   - 예: “저 새끼 학교 앞에서 기다렸다가 한 번 패자”, “쟤 인스타 여기임, 가서 테러하자”  

4. **지속적·집요한 조롱·모욕**
   - 같은 사람을 향한 반복적 모욕, 괴롭힘 패턴이 뚜렷한 경우  

#### WARN_AND_ALLOW 기준

다음은 **게시를 허용하되, 매너·표현 관련 경고를 붙인다.**

1. **욕설·비속어가 있으나 ‘사람’이 아닌 대상**
   - 욕설이 **정책, 제도, 수업, 과제, 학교, 시스템** 등에만 향할 때  
   - 예:  
     - “이 ㅈ같은 등록금 정책 뭐냐 진짜”  
     - “과제 존나 많아서 미치겠음, 학교가 학생을 사람으로 안 봄”  

2. **강한 감정표현·불만, 다소 거친 표현**
   - “개빡친다”, “진짜 미친 거 아님?” 등 **비인칭적 욕설·감탄사** 중심  
   - 특정 개인을 직접 겨냥하지 않고, 상황·경험에 대한 분노 표현  

3. **공적 사안에 대한 날카로운 비판 + 약한 조롱**
   - 교수·학교·학생회 등의 **정책·행동**을 비판하면서 다소 비꼬는 표현이 섞인 경우  
   - 단, “멍청이 교수”, “저 새끼”처럼 **명시적 인신공격**이 되면 BLOCK  

#### ALLOW 기준

다음은 그대로 허용한다.

1. **욕설·비속어가 없는 비판·불만·후기**
   - “수업 퀄리티가 너무 떨어진다”, “과제가 과도하다고 느꼈다”  

2. **정중하거나 중립적인 의견, 정보 공유, 경험담**  

3. **가벼운 농담·슬랭**  
   - 특정인·집단을 비하하지 않는 선의의 농담, 밈 등  
"""

POLICY_B_V2 = """Policy B: Balanced (매너와 토론의 균형)

**Main principles (요약)**  
- 직접적인 인신공격·혐오·위협은 확실히 차단한다.  
- 욕설이 있어도, **정책·제도·수업·상황**을 향한 비판이면 토론 가치를 우선해 허용하되 경고한다.  
- 욕설이 없고 인신공격도 없으면 대부분 허용한다.  
- 모호할 때는 “토론·정보 가치가 있는가?”를 보고, 있으면 WARN_AND_ALLOW, 없으면 BLOCK 쪽으로 기운다.  

#### BLOCK 기준

1. **명시적 인신공격 (개인 대상)**
   - 욕설·비속어·모욕이 **특정 개인**에게 직접 향할 때  
   - 예: “너 진짜 병x냐”, “저 교수는 쓰레기 인간이다”  

2. **혐오 발언 (보호집단 대상)**
   - 성별, 인종, 지역, 성적지향, 장애, 종교 등 보호집단 전체를 모욕·비하  
   - 예: “게이들은 다 정신병자다”, “저 지역 사람은 다 도둑놈”  

3. **폭력·자해·성폭력 등 위협 / 신상털이 / 괴롭힘 선동**
   - 예: “죽여버리고 싶다”, “쟤 이름이랑 학번 여기다 올림, 알아서 해라”  

4. **토론 가치 없이 순수한 모욕·조롱만 있는 경우**
   - 내용·근거 없이 “병신들”, “다 꺼져라” 등만 반복  

#### WARN_AND_ALLOW 기준

1. **욕설·비속어가 있으나, 주 대상이 ‘정책·제도·수업·상황’**
   - 예:  
     - “이 수업 과제량 진짜 미쳤다, 학생들 생각 1도 안 함”  
     - “기숙사 규정 개같이 만들어놨네”  

2. **강한 감정표현 + 논지·경험 공유**
   - 분노·좌절을 표현하면서도, **구체적 이유·경험·논리**가 있는 경우  
   - 예: “이번 학기 진짜 최악이다. 과제가 겹쳐서 잠을 못 잔다. 이런 일정은 개선해야 한다.”  

3. **공적 인물·기관에 대한 거친 비판**
   - 교수·학생회·학교 등 **공적 역할**에 대한 비판에서 다소 비꼬거나 날카로운 표현 사용  
   - 단, “저 새끼”, “저년” 등 노골적 인신공격이면 BLOCK  

#### ALLOW 기준

1. **욕설 없이 표현된 비판·토론·질문**
   - 예: “등록금 인상이 불가피했는지 근거가 궁금합니다.”  

2. **정보 제공, 경험 공유, 건설적 제안**  

3. **감정 표현이 있으나 인신공격·혐오·위협이 없는 경우**
   - 예: “이번 시험 너무 실망스러웠다”, “정말 스트레스 받는다”
"""

POLICY_C_V2 = """Policy C: Minimal Regulation, High Freedom (최소 규제, 표현의 자유 우선)

**Main principles (요약)**  
- 명백한 위협·신상털이·심각한 혐오만 확실히 차단한다.  
- 인신공격·욕설이 있어도, 폭력·신상털이·집단혐오 수준이 아니면 원칙적으로 허용하되 필요시 경고만 한다.  
- 토론·표현의 자유를 최우선으로 두고, 불쾌하더라도 스스로 걸러볼 수 있다고 본다.  
- 모호한 경우에는 BLOCK보다 WARN_AND_ALLOW 또는 ALLOW를 선택한다.  

#### BLOCK 기준

1. **명백한 폭력·자해·성폭력 위협**
   - 특정인 또는 집단에게 **현실적 위해**를 암시하거나 선동  
   - 예: “내일 저 교수 찾아가서 죽여버릴 거다”, “쟤 기숙사 방 번호 아니까 찾아가서 한 번 보자”  

2. **신상털이·표적 괴롭힘 선동**
   - 실명, 학번, 연락처, 계정 등 개인 식별 정보 공개 + 괴롭힘 유도  
   - 예: “이 사람 인스타 @xxx, 가서 욕 좀 해줘라”  

3. **노골적 혐오 발언 (보호집단 대상)**
   - 인종·성별·성적지향·장애·종교·지역 등 보호집단에 대한 **극단적 모욕·비인간화 표현**  
   - 예: “난민은 벌레다, 다 쫓아내야 한다”  

4. **지속적이고 집요한 괴롭힘 패턴**
   - 같은 사람을 반복적으로 지목해 괴롭히는 명백한 패턴  

#### WARN_AND_ALLOW 기준

1. **거친 인신공격·욕설이 있으나, 위협·신상털이·집단혐오까지는 아닌 경우**
   - 예: “저 교수 진짜 개같다”, “너 같은 놈이랑은 말 섞기 싫다”  
   - 불쾌하고 공격적이지만, 현실적 위해나 집단혐오로 이어지지 않는 표현  

2. **강한 조롱·비하가 포함된 논쟁**
   - 토론 중 “말도 안 되는 소리 하지 마라, 수준이 너무 낮다” 등 인신공격성 표현이 섞인 경우  

3. **전반적으로 공격적·무례한 톤**
   - 욕설·비속어가 많고, 읽는 이에게 불편함을 줄 수 있으나, 위 기준의 BLOCK 사유는 아닌 경우  

#### ALLOW 기준

1. **위협·신상털이·집단혐오가 전혀 없는 경우 대부분**
   - 욕설·비속어가 있더라도, 개인 간 감정싸움·불만 표현 수준에 그치는 경우  
   - 예: “이 수업 진짜 ㅈ같다”, “아 너무 빡친다 오늘”  

2. **일반적인 비판·토론·경험 공유**
   - 예: “수업 방식이 나와 잘 안 맞는다”, “등록금 인상은 이해하지만 부담이 크다”  

3. **가벼운 농담·슬랭·친한 사이의 장난**
   - 맥락상 상호 합의된 장난으로 보이고, 심각한 위협·혐오가 없는 경우  
"""

print("Policy A v2 길이:", len(POLICY_A_V2))
print("Policy B v2 길이:", len(POLICY_B_V2))
print("Policy C v2 길이:", len(POLICY_C_V2))

Policy A v2 길이: 1513
Policy B v2 길이: 1239
Policy C v2 길이: 1336


In [6]:
# 5) LLM에 넘길 프롬프트 텍스트 구성

def build_feedback_snippet(df: pd.DataFrame, max_rows: int = 20) -> str:
    """사람 피드백 테이블을 간단한 텍스트 블록으로 변환"""
    rows = []
    subset = df.head(max_rows)
    for _, r in subset.iterrows():
        row_txt = textwrap.dedent(f"""        - sample_id: {r['sample_id']}
          role: {r['role']}, thread_id: {r['thread_id']}
          original_text: {r['text']}
          policy_decisions: A={r['A']}, B={r['B']}, C={r['C']}
          human_preferred_decision: {r['human_preferred_decision']}
          human_comment: {r.get('human_comment', '')}
        """)
        rows.append(row_txt)
    return "\n".join(rows)


feedback_block = build_feedback_snippet(df_feedback, max_rows=30)

system_prompt = """You are an assistant that helps refine content moderation policies for harmful comments/reviews.

- There are three policies: A, B, C.
- Each policy currently defines how to decide one of three outcomes:
  - BLOCK
  - WARN_AND_ALLOW
  - ALLOW

Your task:
- Read the current definitions of Policy A/B/C (v2).
- Read human feedback for some disagreement cases (human-preferred decisions).
- Propose improved Policy A/B/C (v3) descriptions so that:
  - They are still simple enough to implement with an LLM.
  - They better match the human-preferred decisions on these examples.
  - The *relative* philosophy of A/B/C is preserved, roughly:
    - A: strong protection / stricter moderation
    - B: intermediate / emphasize warnings and proportionality
    - C: more permissive / emphasize freedom of expression.

Output format:
- Use Markdown.
- Provide three subsections: 'Policy A v3', 'Policy B v3', 'Policy C v3'.
- For each policy, include:
  - A 3~6 bullet outline of the main principles.
  - Clear guidance on when to choose BLOCK / WARN_AND_ALLOW / ALLOW.
  - If helpful, 2~3 short example patterns of language that should be BLOCKed vs ALLOWed.
- At the end, add a short subsection 'Final recommendation' where you:
  - Recommend which policy (A, B, or C) should be the default 'final' policy after this iteration,
  - Justify your recommendation in 2–4 sentences, using both the stats and qualitative patterns.
"""

user_prompt = f"""## Current policies (v2)

### Policy A v2
{POLICY_A_V2}

### Policy B v2
{POLICY_B_V2}

### Policy C v2
{POLICY_C_V2}

---

## Human feedback on disagreement cases

Below are some cases where policy A/B/C disagreed, and humans provided their preferred decisions.

{feedback_block}

---

## Aggregate agreement stats (human vs policy decisions)

The following are aggregate agreement rates between each policy's decisions (A/B/C) 
and the human_preferred_decision on the feedback dataset:

{stats_block}

---

Please propose refined versions 'Policy A v2', 'Policy B v2', 'Policy C v2' as described in the system instructions.
Also, based on the agreement stats and qualitative patterns, provide a short 'Final recommendation'
section indicating which policy (A, B, or C) is the best candidate to adopt as the final policy after this iteration, and why.
"""

print("=== System prompt preview ===\n")
print(system_prompt)

print("=== User prompt preview ===\n")
print(user_prompt)


=== System prompt preview ===

You are an assistant that helps refine content moderation policies for harmful comments/reviews.

- There are three policies: A, B, C.
- Each policy currently defines how to decide one of three outcomes:
  - BLOCK
  - WARN_AND_ALLOW
  - ALLOW

Your task:
- Read the current definitions of Policy A/B/C (v2).
- Read human feedback for some disagreement cases (human-preferred decisions).
- Propose improved Policy A/B/C (v3) descriptions so that:
  - They are still simple enough to implement with an LLM.
  - They better match the human-preferred decisions on these examples.
  - The *relative* philosophy of A/B/C is preserved, roughly:
    - A: strong protection / stricter moderation
    - B: intermediate / emphasize warnings and proportionality
    - C: more permissive / emphasize freedom of expression.

Output format:
- Use Markdown.
- Provide three subsections: 'Policy A v3', 'Policy B v3', 'Policy C v3'.
- For each policy, include:
  - A 3~6 bullet outline 

In [7]:
# 6) LLM 호출 (정책 v3 제안 받기)

response = client.chat.completions.create(
    model="gpt-5.1",  # 필요에 따라 다른 모델로 변경 가능
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.3,
)

answer = response.choices[0].message.content
print(answer)

# 결과를 파일로 저장
POLICY_V3_PATH = RESULTS_DIR / "policy_v3_suggestions.md"
with open(POLICY_V3_PATH, "w", encoding="utf-8") as f:
    f.write(answer)

print("\n정책 v3 제안 내용을 저장했습니다:")
print(" ->", POLICY_V3_PATH)


### Policy A v3 – High Protection, Low Tolerance (매너 최우선)

**Main principles**

- 사람·집단을 향한 인신공격·혐오는 여전히 강하게 제어하되, “차단”은 진짜 심각한 경우에만 사용한다.  
- 욕설이 없고 인신공격도 없으면, 내용이 마음에 안 들어도 **무조건 ALLOW** 한다.  
- 가벼운 무례·빈정거림·약한 비꼼은 **가능하면 WARN_AND_ALLOW**로 처리하고, BLOCK은 최소화한다.  
- 모호한 경우, “실질적 피해(위협·집단혐오·집요한 괴롭힘)가 있는가?”를 먼저 보고, 없으면 BLOCK 대신 WARN_AND_ALLOW를 우선한다.  

#### BLOCK 기준

다음에 해당할 때만 BLOCK한다. (v2보다 범위를 좁힘)

1. **심각한 인신공격 / 비하 (개인·소수 식별 대상)**  
   - 욕설·비속어·비하가 **특정 개인 또는 소수의 식별 가능한 사람**을 노골적으로 모욕·비인간화할 때  
   - 예:  
     - “저 교수는 인간도 아니다, 진짜 쓰레기다”  
     - “쟤는 장애인 같아 보인다, 존재 자체가 혐오스럽다”  

2. **집단에 대한 혐오·비인간화**  
   - 성별, 지역, 인종, 성적지향, 장애, 종교 등 집단 전체를 **열등·더럽다·벌레 등으로 표현**  
   - 예: “여자는 원래 멍청하다”, “난민은 벌레니까 다 쫓아내야 한다”  

3. **폭력·성폭력·자해 위협 / 신상털이 / 괴롭힘 선동**  
   - 현실적 위해를 암시·선동하거나, 개인 식별 정보 공개 + 공격 유도  
   - 예: “내일 저 교수 찾아가서 죽여버릴 거다”, “쟤 인스타 여기다, 가서 테러하자”  

4. **지속적·집요한 괴롭힘 패턴**  
   - 같은 사람을 반복적으로 지목해 모욕·조롱·따돌림을 유도하는 경우  

> v3 조정 포인트:  
> - “한두 번의 거친 표현·욕설”만으로는 BLOCK하지 않고, 위 기준(심각한 모욕·위협·집단혐오·지속 